In [1]:
bbox = [-94, -88, 27.5, 30.5]

In [1]:
from satpy import Scene, find_files_and_readers
from pyresample import create_area_def
from pyresample.geometry import AreaDefinition

import pyhdf
from satpy.writers import get_enhanced_image
from datetime import datetime
import math

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from glob import glob

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from satpy import MultiScene
from satpy.multiscene import timeseries

%matplotlib inline
import cmocean
import cmocean.cm as cmo

from dask.diagnostics import ProgressBar

ModuleNotFoundError: No module named 'satpy'

In [2]:
from satpy import Scene, find_files_and_readers, available_readers
from glob import glob

## Plot Sea Surface Temperature

Using channel 32

In [3]:
hdf_files = glob('../downloads/*.hdf')
hdf_files

['../downloads\\MOD021KM.A2024245.1550.061.2024247173034.hdf',
 '../downloads\\MOD021KM.A2024245.1555.061.2024247170950.hdf',
 '../downloads\\MOD021KM.A2024246.1630.061.2024247174503.hdf',
 '../downloads\\MOD021KM.A2024246.1635.061.2024247174503.hdf',
 '../downloads\\MOD021KM.A2024247.1535.061.2024248020744.hdf',
 '../downloads\\MOD021KM.A2024247.1710.061.2024248020521.hdf',
 '../downloads\\MOD021KM.A2024248.1615.061.2024249011823.hdf',
 '../downloads\\MOD021KM.A2024249.1515.061.2024250014932.hdf',
 '../downloads\\MOD021KM.A2024249.1655.061.2024250013831.hdf',
 '../downloads\\MOD021KM.A2024250.1555.061.2024251013343.hdf',
 '../downloads\\MOD021KM.A2024250.1600.061.2024251013312.hdf',
 '../downloads\\MOD021KM.A2024251.1500.061.2024252013257.hdf',
 '../downloads\\MOD021KM.A2024251.1635.061.2024252013307.hdf',
 '../downloads\\MOD021KM.A2024251.1640.061.2024252013256.hdf',
 '../downloads\\MOD021KM.A2024252.1540.061.2024253013432.hdf',
 '../downloads\\MOD021KM.A2024252.1715.061.202425301343

In [4]:
available_readers()

C:\Users\Makai\AppData\Roaming\Python\Python312\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


['abi_l1b',
 'abi_l1b_scmi',
 'abi_l2_nc',
 'ahi_hrit',
 'ahi_hsd',
 'ahi_l1b_gridded_bin',
 'ahi_l2_nc',
 'amsr2_l2_gaasp',
 'amsub_l1c_aapp',
 'avhrr_l1b_aapp',
 'avhrr_l1c_eum_gac_fdr_nc',
 'electrol_hrit',
 'ghrsst_l2',
 'gld360_ualf2',
 'glm_l2',
 'goes-imager_hrit',
 'goes-imager_nc',
 'jami_hrit',
 'meris_nc_sen3',
 'mhs_l1c_aapp',
 'mirs',
 'mtsat2-imager_hrit',
 'mviri_l1b_fiduceo_nc',
 'nwcsaf-geo',
 'nwcsaf-pps_nc',
 'olci_l1b',
 'olci_l2',
 'safe_sar_l2_ocn',
 'satpy_cf_nc',
 'seviri_l1b_hrit',
 'seviri_l1b_native',
 'seviri_l1b_nc',
 'slstr_l1b',
 'viirs_edr',
 'viirs_vgac_l1c_nc']

In [5]:
scn = Scene(reader='modis_l1b', filenames=hdf_files)

INFO:satpy.readers:Cannot use ['C:\\Users\\Makai\\AppData\\Roaming\\Python\\Python312\\site-packages\\satpy\\etc\\readers\\modis_l1b.yaml']


ValueError: No supported files found

In [4]:
my_channel = '32'
scn.load([my_channel])
scn[my_channel]

HDF4Error: info (59): Invalid arguments to routine

In [ ]:
area_def = create_area_def('global', {'proj': 'latlong'}, 
                           resolution=0.05)

new_scene = scene.resample(area_def)

In [ ]:
sst = new_scene['32'].data 
fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()})
sst_plot = ax.imshow(sst, cmap=cmo.thermal, transform=ccrs.PlateCarree(),
                     extent=(-94, -88, 27.5, 30.5), origin='upper')

ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.gridlines(draw_labels=True)
plt.colorbar(sst_plot, ax=ax, orientation='vertical', label='Sea Surface Temperature (°C)')

plt.show()

### Map from 0 to 1

In [ ]:
sst_min = sst.min()
sst_max = sst.max()
sst_norm = (sst - sst_min) / (sst_max - sst_min)

fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()})
sst_plot = ax.imshow(sst_norm, cmap=cmo.thermal, transform=ccrs.PlateCarree(),
                     extent=(-94, -88, 27.5, 30.5), origin='upper')

ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.gridlines(draw_labels=True)
plt.colorbar(sst_plot, ax=ax, orientation='vertical', label='Normalized Sea Surface Temperature')

plt.show()

## Plotting CDOM

In [ ]:
hdf_files = glob('../downloads/MOD021KM.A2024238.0225.061.2024238131855.hdf')
scn = Scene(reader='modis_l1b', filenames=hdf_files)
scn.load(['8'])
scn.load(['4'])
scn.load(['13lo'])

lons, lats = scn['8'].attrs['area'].get_lonlats()
crs = ccrs.PlateCarree()

Rss412 = (scn['8'])
Rss555 = (scn['4'])
Rss667 = (scn['13lo'])

B0 = 0.2452
B1 = 5.576
B2 = 4.838
aCDOM355_a = (np.log((Rss412 / Rss555 - B0) / B2)) / (-B1)
aCDOM355_a = aCDOM355_a.compute()

In [ ]:
trim = aCDOM355_a.max().values
fig, ax5 = plt.subplots(subplot_kw={'projection': crs})

ax5.pcolormesh(lons, lats, aCDOM355_a.where(aCDOM355_a <= trim), transform=crs, vmin=0.01, vmax=0.3)
ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.gridlines(draw_labels=True)
plt.colorbar(sst_plot, ax=ax, orientation='vertical', label='Sea Surface Temperature (°C)')
ax5.set_extent([-94, -88, 27.5, 30.5], crs=crs)

ax5.set_title('CDOM Concentration')

plt.show()

Mapping

In [ ]:
min_val = aCDOM355_a.min().values
max_val = aCDOM355_a.max().values
aCDOM355_a_normalized = (aCDOM355_a - min_val) / (max_val - min_val)

trim = aCDOM355_a_normalized.max().values

fig, ax5 = plt.subplots(subplot_kw={'projection': crs})
sst_plot = ax5.pcolormesh(lons, lats, aCDOM355_a_normalized.where(aCDOM355_a_normalized <= trim), 
                          transform=crs, vmin=0, vmax=1)
ax5.coastlines()
ax5.add_feature(cfeature.BORDERS, linestyle=':')
ax5.gridlines(draw_labels=True)

plt.colorbar(sst_plot, ax=ax5, orientation='vertical', label='Normalized CDOM Concentration (0-1)')
ax5.set_extent([-94, -88, 27.5, 30.5], crs=crs)
ax5.set_title('Normalized CDOM Concentration')

plt.show()

# Final Code

In [ ]:
area_def = create_area_def('global', {'proj': 'latlong'}, resolution=0.05,
                           area_extent=(-94, -88, 27.5, 30.5))

new_scene_cdom = scn.resample(area_def)
aCDOM355_a_resampled = new_scene_cdom['8'].data.compute()

new_scene_sst = scn.resample(area_def)
sst_resampled = new_scene_sst['32'].data.compute()

In [ ]:
cdom_min = aCDOM355_a_resampled.min()
cdom_max = aCDOM355_a_resampled.max()
aCDOM355_a_normalized = (aCDOM355_a_resampled - cdom_min) / (cdom_max - cdom_min)

sst_min = sst_resampled.min()
sst_max = sst_resampled.max()
sst_normalized = (sst_resampled - sst_min) / (sst_max - sst_min)

assert aCDOM355_a_normalized.shape == sst_normalized.shape, "Shapes are not aligned!"

In [ ]:
combined_map = aCDOM355_a_normalized * sst_normalized

fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree()})
combined_plot = ax.imshow(combined_map, cmap='viridis', transform=ccrs.PlateCarree(),
                          extent=(-94, -88, 27.5, 30.5), origin='upper')

ax.coastlines()
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.gridlines(draw_labels=True)

# Add color bar and labels
plt.colorbar(combined_plot, ax=ax, orientation='vertical', label='Combined Map (Normalized)')
ax.set_title('Combined CDOM and SST Map')

plt.show()

## Verfy all available channels

In [ ]:
# hdf_file = '../downloads/MOD021KM.A2024238.0225.061.2024238131855.hdf'
scene = Scene(reader='modis_l1b', filenames=hdf_files)
print(scene.available_dataset_names())
